In [7]:
import os
from datetime import datetime, timedelta
import pandas as pd
from zeep import Client
import loadenv
import db
from db import get_connection
import psycopg2.extras
import holidays

AGS_TO_ISO = {
    '01': 'SH',
    '02': 'HH',
    '03': 'NI',
    '04': 'HB',
    '05': 'NW',
    '06': 'HE',
    '07': 'RP',
    '08': 'BW',
    '09': 'BY',
    '10': 'SL',
    '11': 'BE',
    '12': 'BB',
    '13': 'MV',
    '14': 'SN',
    '15': 'ST',
    '16': 'TH'
}

In [20]:
conn, cur = get_connection()

def process_county_ebrake(county_id) -> None:
    cur.execute(f"""
        SELECT timestamp, \"7_day_incidence\" 
        FROM rki_incidence_excel_berlin 
        WHERE ags = '{county_id}'
        AND datenbestand = (SELECT MAX(datenbestand) FROM rki_incidence_excel)
        AND timestamp >= '2021-04-20' ORDER BY timestamp""")
    c_data = cur.fetchall()

    # delete existing ebrake data
    cur.execute(f"DELETE FROM counties_ebrake WHERE id = '{county_id}'")

    # get holidays for the state of the county:
    state_holidays = holidays.Germany(prov=AGS_TO_ISO[county_id[0:2]])

    ret_data = []
    for d in c_data:
        ret_data.append({
            'id': county_id,
            'ts': d[0],
            'val': round(d[1], 2),
            'ebrake100': False,
            'ebrake150': False,
            'ebrake165': False,
            'holiday': state_holidays.get(d[0])
        })

    # nowcast
    today = datetime.combine(datetime.today(), datetime.min.time())
    if ret_data[-1]['ts'] < today:
        ret_data.append({
            'id': county_id,
            'ts': today,
            'val': None,
            'ebrake100': False,
            'ebrake150': False,
            'ebrake165': False,
            'holiday': state_holidays.get(d[0])
        })

    # forecast
    for i in range(1, 8):
        future_dt = today + timedelta(days=i)
        ret_data.append({
            'id': county_id,
            'ts': future_dt,
            'val': None,
            'ebrake100': False,
            'ebrake150': False,
            'ebrake165': False,
            'holiday': state_holidays.get(d[0])
        })
        
    # contains the idx when ebrake has started for respective threshold
    ebrake_start = {100: None, 150: None, 165: None}
    for i in range(4, len(ret_data)):

        # check for date idx = i if ebrake has started
        # must be over t for 3 days
        ret_data[i]['ebrake100'] = True
        ret_data[i]['ebrake150'] = True
        ret_data[i]['ebrake165'] = True
        skipped = False
        for j in range(i - 4, i - 1):
            if ret_data[j]['val'] is None:
                skipped = True
                continue

            if ret_data[j]['val'] < 165:
                ret_data[i]['ebrake165'] = False
                
            if ret_data[j]['val'] < 150:
                ret_data[i]['ebrake150'] = False

            if ret_data[j]['val'] < 100:
                ret_data[i]['ebrake100'] = False
                break

        if skipped is True:
            ret_data[i]['ebrake100'] = None
            ret_data[i]['ebrake150'] = None
            ret_data[i]['ebrake165'] = None

        if ret_data[i]['ebrake100'] is True and ebrake_start[100] is None:
            ebrake_start[100] = i
            
        if ret_data[i]['ebrake150'] is True and ebrake_start[150] is None:
            ebrake_start[150] = i

        if ret_data[i]['ebrake165'] is True and ebrake_start[165] is None:
            ebrake_start[165] = i

        # date is still in eBrake
        if ebrake_start[100] is not None:
            ret_data[i]['ebrake100'] = True

        if ebrake_start[150] is not None:
            ret_data[i]['ebrake150'] = True
            
        if ebrake_start[165] is not None:
            ret_data[i]['ebrake165'] = True

        # only necessary if currently in ebrake
        if ebrake_start[100] is not None:
            over100 = None
            over150 = None
            over165 = None
            num_weekdays = 0
            # start with -2 because it only concerns the day after tomorrow
            j = i - 2
            # go back in time until 5 weekdays are processed or beginning of data is reached
            while j >= 0 and num_weekdays < 5:
                # sunday and holidays are skipped
                if ret_data[j]['ts'].isoweekday() == 7 \
                        or ret_data[j]['holiday'] is not None:
                    j -= 1
                    continue

                # in case incidence is not available because of future predictions
                if ret_data[j]['val'] is None:
                    j -= 1
                    # still count this as a weekday
                    num_weekdays += 1
                    continue

                if ebrake_start[100] is not None and (ret_data[j]['val'] >= 100 or j < (ebrake_start[100] + 1)):
                    over100 = True
                elif over100 is None:
                    over100 = False
                    
                if ebrake_start[150] is not None and (ret_data[j]['val'] >= 150 or j < (ebrake_start[150] + 1)):
                    over150 = True
                elif over150 is None:
                    over150 = False

                if ebrake_start[165] is not None and (ret_data[j]['val'] >= 165 or j < (ebrake_start[165] + 1)):
                    over165 = True
                    break
                elif over165 is None:
                    over165 = False

                num_weekdays += 1
                j -= 1

            if over165 is False:
                ret_data[i]['ebrake165'] = False
                ebrake_start[165] = None
            elif over165 is None:
                ret_data[i]['ebrake165'] = None
                ebrake_start[165] = None
                
            if over150 is False:
                ret_data[i]['ebrake150'] = False
                ebrake_start[150] = None
            elif over150 is None:
                ret_data[i]['ebrake150'] = None
                ebrake_start[150] = None

            if over100 is False:
                ret_data[i]['ebrake100'] = False
                ebrake_start[100] = None
            elif over100 is None:
                ret_data[i]['ebrake100'] = None
                ebrake_start[100] = None
    
    for d in ret_data:
        print(d)

    # write to DB:
    psycopg2.extras.execute_values(
        cur,
        "INSERT INTO counties_ebrake (id, timestamp, ebrake100, ebrake150, ebrake165, holiday) VALUES %s",
        ret_data,
        template='(%(id)s, %(ts)s, %(ebrake100)s, %(ebrake150)s, %(ebrake165)s, %(holiday)s)',
        page_size=500
    )
    conn.commit()

    

        
process_county_ebrake('11000')
process_county_ebrake('08335')
process_county_ebrake('02000')

cur.execute("SELECT ids FROM landkreise_extended GROUP BY ids ORDER BY ids")
county_ids = cur.fetchall()


# for c in county_ids:
#     process_county_ebrake(c[0])

{'id': '11000', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': Decimal('145.69'), 'ebrake100': False, 'ebrake150': False, 'ebrake165': False, 'holiday': None}
{'id': '11000', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': Decimal('135.71'), 'ebrake100': False, 'ebrake150': False, 'ebrake165': False, 'holiday': None}
{'id': '11000', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': Decimal('133.83'), 'ebrake100': False, 'ebrake150': False, 'ebrake165': False, 'holiday': None}
{'id': '11000', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': Decimal('134.95'), 'ebrake100': False, 'ebrake150': False, 'ebrake165': False, 'holiday': None}
{'id': '11000', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': Decimal('132.63'), 'ebrake100': True, 'ebrake150': False, 'ebrake165': False, 'holiday': None}
{'id': '11000', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': Decimal('135.41'), 'ebrake100': True, 'ebrake150': False, 'ebrake165': False, 'holiday': None}
{'id': '11000', 'ts': da

In [ ]:
conn, cur = get_connection()

cur.execute("SELECT county_id FROM cases_per_county_and_day GROUP BY county_id")
county_ids = cur.fetchall()


for c in county_ids:
    process_county(c[0])
    